<a href="https://colab.research.google.com/github/online6731/COVID-19-Dashboard/blob/main/COVID_19_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment

In [1]:
#@title Import + Config { vertical-output: true }

! pip install plotly -Uq
! pip install jupyter-dash -q
! pip install dash-bootstrap-components -q

import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
from google.colab.data_table import DataTable

pio.templates.default = 'plotly'
pd.options.plotting.backend = "plotly"

     |████████████████████████████████| 13.2MB 305kB/s 
     |████████████████████████████████| 81kB 3.3MB/s 
     |████████████████████████████████| 1.0MB 26.2MB/s 
     |████████████████████████████████| 3.5MB 36.2MB/s 
     |████████████████████████████████| 92kB 8.2MB/s 
     |████████████████████████████████| 1.8MB 35.3MB/s 
     |████████████████████████████████| 358kB 42.8MB/s 
     |████████████████████████████████| 194kB 4.3MB/s 


# Dataset

In [2]:
#@title Data
#@markdown https://github.com/owid/covid-19-data

df = pd.read_csv('https://github.com/owid/covid-19-data/raw/master/public/data/owid-covid-data.csv')

DataTable(df.sample(100), 
          num_rows_per_page=5, 
          max_columns=100)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,new_vaccinations_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
368,AFG,Asia,Afghanistan,2021-02-26,55696.0,16.0,17.286,2442.0,4.0,1.714,1430.731,0.411,0.444,62.731,0.103,0.044,0.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,12.04,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
24422,DMA,North America,Dominica,2021-03-08,144.0,0.0,0.286,NaN,NaN,0.000,2000.250,0.000,3.969,NaN,NaN,0.000,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12275.0,12275.0,NaN,NaN,725.0,17.05,17.05,NaN,10071.0,29.63,71991.0,98.567,NaN,NaN,NaN,9673.367,NaN,227.376,11.62,NaN,NaN,NaN,3.8,75.00,0.742,NaN
21753,CYP,Europe,Cyprus,2020-04-03,396.0,40.0,33.429,11.0,1.0,0.857,452.107,45.667,38.165,12.559,1.142,0.979,1.25,12.0,13.7,55.0,62.793,NaN,NaN,NaN,NaN,668.0,9447.0,10.785,0.763,NaN,NaN,NaN,NaN,tests performed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.59,875899.0,127.657,37.3,13.416,8.563,32415.132,NaN,141.171,9.24,19.6,52.7,NaN,3.4,80.98,0.887,NaN
45436,KAZ,Asia,Kazakhstan,2021-05-19,420863.0,2662.0,2359.286,3409.0,6.0,3.143,22414.101,141.771,125.650,181.555,0.320,0.167,0.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50238.0,11065190.0,589.304,2.676,53440.0,2.846,0.044,22.7,tests performed,2700694.0,1898604.0,802090.0,50525.0,32282.0,14.38,10.11,4.27,1719.0,62.96,18776707.0,6.681,30.6,6.991,4.625,24055.588,0.1,466.792,7.11,7.0,43.1,98.999,6.7,73.60,0.825,NaN
92757,VEN,South America,Venezuela,2020-09-02,48883.0,1127.0,988.286,398.0,7.0,6.714,1719.057,39.633,34.755,13.996,0.246,0.236,1.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.41,28435943.0,36.253,29.0,6.614,3.915,16745.022,NaN,204.850,6.47,NaN,NaN,NaN,0.8,72.06,0.711,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41619,IRQ,Asia,Iraq,2020-09-15,298702.0,4224.0,4160.571,8166.0,80.0,72.714,7426.241,105.016,103.439,203.021,1.989,1.808,0.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22141.0,1948531.0,48.444,0.550,22580.0,0.561,0.184,5.4,samples tested,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.15,40222503.0,88.125,20.0,3.186,1.957,15663.986,2.5,218.612,8.83,NaN,NaN,94.576,1.4,70.60,0.674,NaN
19656,COG,Africa,Congo,2021-02-14,8419.0,0.0,51.286,123.0,0.0,0.143,1525.709,0.000,9.294,22.290,0.000,0.026,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.52,5518092.0,15.405,19.0,3.402,2.063,4881.406,37.0,344.094,7.20,1.7,52.3,47.964,NaN,64.57,0.574,NaN
74936,SEN,Africa,Senegal,2020-06-09,4516.0,89.0,97.143,52.0,3.0,1.286,269.710,5.315,5.802,3.106,0.179,0.077,1.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# Charts

In [3]:
#@title Death vs Cases per Continent { vertical-output: true }

continent_death_case = px.bar(
    df.groupby('continent').sum().sort_values('new_cases'), 
    x=['new_deaths', 'new_cases'],
    orientation='h',
    barmode='group',
    # title='Death vs Cases per Continent'
    ).update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                     'paper_bgcolor': 'rgba(0, 0, 0, 0)'})

continent_death_case

In [4]:
#@title Total Cases per Continent { vertical-output: true }

continent_case = px.pie(
    df.groupby('continent').sum().reset_index(),
    values='new_cases',
    names='continent',
    hover_name='continent',
    # title='Total Cases per Continent'
    ).update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                     'paper_bgcolor': 'rgba(0, 0, 0, 0)'})

continent_case

In [5]:
#@title Cases vs Deaths over Time { vertical-output: true }

case_death_over_time = px.line(
    df.groupby('date').sum().reset_index(), 
    x='date',
    y=['new_cases', 'new_deaths'],
    # title='Cases vs Deaths over Time'
    ).update_xaxes(rangeslider_visible=True
    ).update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)',
                    'paper_bgcolor': 'rgba(0, 0, 0, 0)'})

case_death_over_time

In [6]:
#@title Stats { vertical-output: true }

stats = {
    'Overal Cases': df['new_cases'].sum(),
    'Overal Deaths': df['new_deaths'].sum(),
    'Today Cases': df[df['date'] == df['date'].to_list()[-1]]['new_cases'].sum(),
    'Today Deaths': df[df['date'] == df['date'].to_list()[-1]]['new_deaths'].sum()
}

pd.DataFrame.from_dict(stats, 
                       orient='index', 
                       columns=['Stats'])

,Stats
Overal Cases,559164724.0
Overal Deaths,12134900.0
Today Cases,766813.0
Today Deaths,16809.0


# Dashboard

In [9]:
#@title Dashoard { vertical-output: true, form-width: "20%" }

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.FLATLY])

app.layout = html.Div([
    html.Header([
            html.H1('COVID-19'),
            html.Div([
                html.H4('Dashboard'),
                html.H4('Analytics'),
                html.H4('News'),
                html.H4('About'),
            ], className='d-flex col-6 justify-content-around align-self-center')
    ], className='d-flex border-bottom m-4 justify-content-between'),
    
    html.Div([
        dbc.Card([
            dbc.CardHeader(label, className='h5'),
            dbc.CardBody(stats[label], className='h3'),
        ], className='bg-light p-1 col-2 text-center')
        for label in ['Overal Cases', 'Today Cases', 'Overal Deaths', 'Today Deaths']
    ], className='d-flex justify-content-between m-4'),
    
    html.Div([
              dbc.Card([dbc.CardHeader('Cases per Continent', className='h5'),
                        dbc.CardBody(dcc.Graph(figure=continent_case))
                        ], className='bg-light p-1 col-4'),
              
              dbc.Card([dbc.CardHeader('Cases vs Deaths over Time', className='h5'),
                        dbc.CardBody(dcc.Graph(figure=continent_death_case))
                        ], className='bg-light p-1 col-7'),
              
              ], className='d-flex justify-content-between m-4'),
    
    dbc.Card([
              dbc.CardHeader('Cases vs Deaths over Time', className='h5'),
              dbc.CardBody(dcc.Graph(figure=case_death_over_time))
              ], className='d-flex justify-content-between m-4 bg-light')

], className='d-flex flex-column mx-5 my-2')

# ! fuser -k 8050/tcp

app.run_server(mode='external', debug=True)

Dash app running on:


<IPython.core.display.Javascript object>